In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 22 10:40:56 2020

@author: marcelo

ar1:
- array (nx,2), onde:
    nx: n'umero total de picos em todas as escalas
    x: 'indice do pico
    ar1[x,0]: canal inteiro do pico
    ar1[x,1]: valor da escala
    
xpk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise
    x: canal inteiro do pico
    
ypk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise (= nx(xpk))
    x: valor da escala

"""

import numpy as np
import numpy.ma as ma  # masked array
from numpy import linalg as LA

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from gamma_spectrum_class import (Spec)

from machine_selection import set_screen_size_by_machine
from bokeh.models import ColumnDataSource, Whisker, Label
from bokeh.palettes import Paired6
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot, column, row

from bokeh import palettes

# 2020-08-22 Para rodar curve_fit
from gauss_funcs import func_example, func_example_fixed_b
# 2020-09-03 Desativei por enquanto
# import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# 2020-08-30
from scipy.interpolate import splrep, splev

from scipy.ndimage import label, generate_binary_structure, find_objects, binary_dilation  # 2020-09-03
from numpy.polynomial import Polynomial as P # 2020-09-06 Esta é a nova classe recomendada 

In [2]:
# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

In [3]:
def stepBaseLine(ch0, ch1, y0, y1, ys):
    contin = []
    G=np.sum(ys[ch0:ch1])
    deltaY = y1-y0
    deltaX = ch1-ch0
    dydx = deltaY/deltaX
    for i in range(ch0,ch1+1):
        sumY = np.sum(ys[ch0:i])
        contin.append( y0 + deltaY * sumY/G )
    return contin

In [4]:
hei = 0.01

# haWin = 3
# yrange = [10**1, 10**5]
# fn2 = '../Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'

# haWin = 3 
# yrange = [10**1, 10**5]
# fn2 = '../Genie_Transfer/Si/SI2019/SI04019.Chn'

# haWin = 6
# fn2 = '../Pesquisa/2020_Mariana_PR/2020_Mariana_PR_Espectros/espectros-det-projeto/agua-bruta.IEC'

haWin = 6
yrange = [10**1, 10**5]
fn2 = '../gamma/spectra/ALMERA-2016/alm_samp5_F100_16_Cont-D_133k.IEC'

# haWin = 6
# yrange = [10**1, 10**6]
# fn2 = './I125_01.001.3.IEC'

# 13 de julho de 2020
# PAREI AQUI O termo de 1o grau oscila selvagemente em baixas contagens
# ponderá-lo pelo coeffs0



In [5]:
spec2 = Spec()
if spec2.readSp( fn2 ) >= 0:
    print (spec2.nCh)
    
counts = spec2.specIO.spCounts
print(len(counts))
# smo0, smo1, sum1, bline = spec2.coeffs_2nd_grade_estim(haWin=haWin)

spec2.coeffs_Calibs()

print ('\nChannel vs Energy')
print (spec2.enChcalib)
print (spec2.chancalib)
print (spec2.coeffs_ChEn)

print ('\nEnergy vs FWHM')
print (spec2.enFwcalib)
print (spec2.fwhmcalib)
print (spec2.coeffs_EnFw)

print ('\nEnergy vs Efficiency')
print (spec2.enEfcalib)
print (spec2.efficalib)
print (spec2.coeffs_EnEf)

8192
8192

Channel vs Energy
[  46.54   338.32   609.312 1120.29  1460.81 ]
[ 236.289 1712.41  3083.95  5672.44  7398.92 ]
[-4.00151430e-08  1.97757610e-01 -1.91052541e-01]

Energy vs FWHM
[  46.54   338.32   609.312 1120.29  1460.81 ]
[ 4.14796  5.47718  6.5903   8.72066 10.6854 ]
[ 1.33173266e-09 -2.45953829e-06  5.39601735e-03  3.89756470e+00]

Energy vs Efficiency
[  30.85    53.161   80.997  276.398  302.853  356.017  383.851  563.227
  569.315  604.699  621.84   661.66   795.845  801.932 1115.55  1173.24
 1332.5  ]
[0.04078 0.05937 0.05247 0.02347 0.02246 0.02087 0.02205 0.01468 0.01441
 0.01305 0.01328 0.01391 0.01052 0.01157 0.00829 0.00807 0.00723]
[-5.50950773e-11  1.53992636e-07 -1.45591433e-04  5.69774291e-02]


In [6]:
print (spec2.chancalib)

[ 236.289 1712.41  3083.95  5672.44  7398.92 ]


In [7]:
len(spec2.chancalib)

5

In [8]:
# 2020-08-30 AQUI: Detecção dos picos

In [9]:
imag, pksCh, nscpk, chanCwtpk, iscaleCwtpk, maxcoefCwtpk, widths = spec2.pks_by_ridges(
    spec2.y0s, widths_range=(3,20), msnr=2.0, minCoefCwt=30.0)  
# Finalmente achei o parâmetro crítico: window_size. O default é muito grande!

In [10]:
# 2020-08-31 PAREI AQUI: O pksCh obtido diretamente de find_pks_cwt não dá resultados bons;
# Picos importantes, próximos de picos maiores, não são detectados, mesmo com os kwargs
# bem ajustados, o que prejudica a delimitação das regiões
# Vou usar os ridges criados com a análise de imagem, para definir as regiões por meio dos retângulos
# e buscando o máximo cwt em cada ridge para selecionar os picos.

In [11]:
indices_tupl = imag.nonzero()
np.asarray(indices_tupl).shape

(2, 2084)

In [12]:
# 2020-08-30 Perfeito! É o que eu queria para listar indices dos elementos não nulos

In [13]:
nChPart = spec2.lnCounts.size
nChPart

8192

In [14]:
# 2020-08-30 Bokeh graph 1
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    plot_height=300,
    # plot_height=default_plot_height,
    tools=tools
)
float_formatter = "{:.2f}".format
for i in range(len(chanCwtpk)):
    mytext = Label(x=chanCwtpk[i], y=iscaleCwtpk[i], text=float_formatter(maxcoefCwtpk[i]),
                   text_font_size='10pt')
    boka.add_layout(mytext)
boka.circle(indices_tupl[1], indices_tupl[0], size=3)

bokb = figure(
    plot_width =default_plot_width,
    plot_height=300,
    y_axis_type="log",
    tools=tools
)
float_formatter = "{:n}".format
for i in pksCh:
    mytext = Label(x=i, y=spec2.y0s[i], text=float_formatter(i), text_font_size='10pt')
    bokb.add_layout(mytext)

bokb.x_range = boka.x_range
bokb.circle(spec2.xs, spec2.y0s, color="red",size=2)
bokb.line(spec2.xs, spec2.y0s, color="yellow")
bokb.circle(pksCh, spec2.y0s[pksCh], size=8, line_color='green', fill_color='white', fill_alpha=0)

# set up layout
p = gridplot([boka, bokb], ncols=1)
show(p)

In [15]:
chanCwtpk.size

124

In [16]:
len(iscaleCwtpk)

124

In [17]:
# 2020-08-30 Bokeh graph 2
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    plot_height=300,
    # plot_height=default_plot_height,
    tools=tools
)
float_formatter = "{:.2f}".format
boka.circle(indices_tupl[1], indices_tupl[0], size=3)
boka.circle(chanCwtpk, iscaleCwtpk, size=8, line_color='magenta', fill_color='white', fill_alpha=0)

bokb = figure(
    plot_width =default_plot_width,
    plot_height=300,
    y_axis_type="log",
    tools=tools
)
float_formatter = "{:n}".format
#for i in pksCh:
#    mytext = Label(x=i, y=spec2.y0s[i], text=float_formatter(i), text_font_size='10pt')
#    bokb.add_layout(mytext)
for i in range(len(chanCwtpk)):
    mytext = Label(x=chanCwtpk[i], y=spec2.y0s[chanCwtpk][i], text=float_formatter(maxcoefCwtpk[i]),
                   text_font_size='10pt')
    bokb.add_layout(mytext)
bokb.circle(chanCwtpk, spec2.y0s[chanCwtpk], size=8, line_color='blue', fill_color='white', fill_alpha=0)
    
bokb.x_range = boka.x_range
bokb.circle(spec2.xs, spec2.y0s, color="red",size=2)
bokb.line(spec2.xs, spec2.y0s, color="yellow")

# set up layout
p = gridplot([boka, bokb], ncols=1)
show(p)

In [18]:
float_formatter = "{:.2f}".format

In [19]:
widths

array([ 3.        ,  3.5862069 ,  4.17241379,  4.75862069,  5.34482759,
        5.93103448,  6.51724138,  7.10344828,  7.68965517,  8.27586207,
        8.86206897,  9.44827586, 10.03448276, 10.62068966, 11.20689655,
       11.79310345, 12.37931034, 12.96551724, 13.55172414, 14.13793103,
       14.72413793, 15.31034483, 15.89655172, 16.48275862, 17.06896552,
       17.65517241, 18.24137931, 18.82758621, 19.4137931 , 20.        ])

In [20]:
scales = np.asarray(iscaleCwtpk)

In [21]:
scales

array([ 3,  0,  0,  1,  0,  4,  0,  0,  3,  0,  0,  0,  0,  1,  2,  2,  4,
        2,  1,  0,  0,  3,  0,  3,  4,  4,  3,  4,  2,  0,  4,  1,  2,  2,
        0,  3,  3,  0,  0,  3,  0,  1,  1,  2,  0,  1,  0,  0,  0,  4,  1,
        0,  3,  0,  4,  4,  4,  1,  0,  4,  0,  2,  2,  3,  2,  0,  2,  3,
        3,  1,  2,  4,  4,  2,  3,  3,  4,  4,  4,  0,  3,  4,  3,  3,  3,
        3,  3,  3,  3,  4,  2,  3,  3,  3,  3,  4,  4,  3,  3,  3,  2,  3,
        2,  2,  2,  2,  0,  2,  2,  1,  2,  1,  1,  2,  4,  4,  7,  7,  8,
        8,  9, 12, 15, 16])

In [22]:
chanCwtpk

array([ 112,  130,  140,  163,  187,  205,  237,  255,  271,  304,  321,
        343,  391,  426,  443,  470,  500,  535,  552,  572,  621,  654,
        705,  729,  759,  781,  828,  862,  883,  909,  943,  969,  993,
       1008, 1022, 1040, 1060, 1122, 1209, 1226, 1267, 1297, 1311, 1366,
       1373, 1391, 1405, 1436, 1450, 1495, 1519, 1618, 1640, 1660, 1690,
       1713, 1782, 1901, 1958, 1969, 1993, 2034, 2051, 2073, 2119, 2163,
       2251, 2303, 2343, 2378, 2400, 2433, 2466, 2586, 2673, 2702, 2953,
       3085, 3369, 3484, 3560, 3645, 3682, 3760, 3890, 3979, 4025, 4082,
       4183, 4212, 4230, 4249, 4358, 4614, 4730, 4884, 4907, 5070, 5328,
       5674, 5851, 6117, 6271, 6489, 6979, 7018, 7099, 7132, 7400, 7644,
       8019,  379, 2936, 1671,  912,  806,  167,  206,  204, 1038, 1396,
        619,  272, 1441])

In [23]:
kWin = 3.5

In [24]:
# 2020-09-05 Fator para cálculo da largura da janela: kWin

In [25]:
# https://stackoverflow.com/questions/10873824/how-to-convert-2d-float-numpy-array-to-2d-int-numpy-array
halfWins = np.int_(np.rint(kWin*widths[scales]))
halfWins

array([17, 10, 10, 13, 10, 19, 10, 10, 17, 10, 10, 10, 10, 13, 15, 15, 19,
       15, 13, 10, 10, 17, 10, 17, 19, 19, 17, 19, 15, 10, 19, 13, 15, 15,
       10, 17, 17, 10, 10, 17, 10, 13, 13, 15, 10, 13, 10, 10, 10, 19, 13,
       10, 17, 10, 19, 19, 19, 13, 10, 19, 10, 15, 15, 17, 15, 10, 15, 17,
       17, 13, 15, 19, 19, 15, 17, 17, 19, 19, 19, 10, 17, 19, 17, 17, 17,
       17, 17, 17, 17, 19, 15, 17, 17, 17, 17, 19, 19, 17, 17, 17, 15, 17,
       15, 15, 15, 15, 10, 15, 15, 13, 15, 13, 13, 15, 19, 19, 25, 25, 27,
       27, 29, 35, 41, 43])

In [26]:
xbl = np.ones(nChPart,dtype=int)
for i,pk in enumerate(chanCwtpk):
    xbl[pk-halfWins[i]:pk+halfWins[i]+1] = 0
imabl = np.array([xbl,xbl,xbl])

In [27]:
stru22 = generate_binary_structure(2,2)
lblimag22, num_feat_imag22 = label(imabl,stru22)
objs = find_objects(lblimag22)
stopsBls = [i[1].stop for i in objs]
startsBls = [i[1].start for i in objs]
len(objs)

55

In [28]:
smoo=11

In [29]:
splTcks = []
for obj in objs:
    m = spec2.xs[obj[1]].size
    if m > 3:
        splTcks.append(splrep(spec2.xs[obj[1]], spec2.lnCounts[obj[1]], k=3, s=smoo))

In [30]:
xBlPlot = []
yBlPlot = []
for tck in splTcks:
    xIn = tck[0][0]
    xFi = tck[0][-1]
    xBlPl = np.linspace(xIn, xFi, num=int(np.round(3*(xFi-xIn))))
    xBlPlot.append( xBlPl )
    yBlPlot.append( np.exp(splev(xBlPl, tck, der=0)) )

In [31]:
xRgPlot = []
yRgPlot = []
for i in range(len(splTcks)-1):
    xIn = splTcks[i][0][-1]
    xFi = splTcks[i+1][0][1]
    y0 = np.exp(splev([int(xIn)], splTcks[i])[0])
    y1 = np.exp(splev([int(xFi)], splTcks[i+1])[0])
    xRgPlot.append( np.linspace(xIn, xFi, num=int(xFi-xIn)+1) )
    yRgPlot.append( stepBaseLine(int(xIn), int(xFi), y0, y1, spec2.y0s) )

In [32]:
# Tentar aqui um método mais direto para estimar baseline:
blineMin = np.zeros(nChPart)
for i in range(15,nChPart):
    blineMin[i] = np.amin(spec2.y0s[i-15:i+15+1])

In [33]:
# Tentar aqui um método mais direto para estimar baseline: esta está muito bom!!
blineMin = np.zeros(nChPart)
for i in range(18,nChPart):
    blineMin[i] = np.percentile(spec2.y0s[i-18:i+18+1], 10.0)

In [34]:
smoo

11

In [35]:
# Tentar aqui um método mais direto para estimar baseline: esta está melhor ainda!!!!!
# Aqui usando spline SOBRE LNCOUNTS
blineMin = np.zeros(nChPart)
for i in range(18,nChPart):
    blineMin[i] = np.percentile(spec2.lnCounts[i-18:i+18+1], 10.0)
tckDirectSpl = splrep(spec2.xs, blineMin, k=3, s=smoo)
directSpl = np.exp(splev(spec2.xs, tckDirectSpl, der=0))

In [36]:
xBlPl.size

468

In [37]:
# 2020-09-06 Mais um método para estimar baseline: fazer percentil em ajuste
# quadrático em janelas do logaritmo das contagens

# https://numpy.org/doc/stable/reference/routines.polynomials.classes.html

matrFits = np.zeros((nChPart, 37))
blineQuaPerc = np.zeros(nChPart)
for i in range(18,nChPart-19):
    p = P.fit(spec2.xs[i-18:i+18+1], spec2.lnCounts[i-18:i+18+1], deg=2)
    matrFits[i,:] = p(spec2.xs[i-18:i+18+1])
for i in range(18,nChPart-19):
    dia = np.fliplr(matrFits[i-18:i+18+1,:]).diagonal()  # Horizontal flip
    blineQuaPerc[i] = np.exp(np.percentile(dia, 10.0))

In [38]:
# 2020-09-06 Mais um método para estimar baseline: fazer percentil em ajuste
# quadrático em janelas do logaritmo das contagens

# https://numpy.org/doc/stable/reference/routines.polynomials.classes.html

matrFits = np.zeros((nChPart, 37))
blineLinPerc = np.zeros(nChPart)
for i in range(18,nChPart-19):
    p = P.fit(spec2.xs[i-18:i+18+1], spec2.lnCounts[i-18:i+18+1], deg=1)
    matrFits[i,:] = p(spec2.xs[i-18:i+18+1])
for i in range(18,nChPart-19):
    dia = np.fliplr(matrFits[i-18:i+18+1,:]).diagonal()  # Horizontal flip
    blineLinPerc[i] = np.exp(np.percentile(dia, 10.0))

In [39]:
# 2020-09-06 Mais um método para estimar baseline: fazer percentil em ajuste
# quadrático em janelas do logaritmo das contagens

# https://numpy.org/doc/stable/reference/routines.polynomials.classes.html

hWin = 25
matrFits = np.zeros((nChPart, 2*hWin+1))
blineConPerc = np.zeros(nChPart)
for i in range(hWin,nChPart-hWin-1):
    p = P.fit(spec2.xs[i-hWin:i+hWin+1], spec2.lnCounts[i-hWin:i+hWin+1], deg=0)
    matrFits[i,:] = p(spec2.xs[i-hWin:i+hWin+1])
for i in range(hWin,nChPart-hWin-1):
    dia = np.fliplr(matrFits[i-hWin:i+hWin+1,:]).diagonal()  # Horizontal flip
    blineConPerc[i] = np.exp(np.percentile(dia, 5.0))

In [40]:
splrep(spec2.xs[obj[1]], spec2.lnCounts[obj[1]], k=3, s=smoo)

(array([8035., 8035., 8035., 8035., 8055., 8074., 8094., 8113., 8123.,
        8133., 8152., 8162., 8172., 8191., 8191., 8191., 8191.]),
 array([3.05721223, 3.67795138, 2.73305538, 3.63173962, 2.9783063 ,
        3.05385731, 2.63715145, 2.90865429, 2.98127897, 2.56739887,
        2.7257536 , 2.96282995, 2.80932372, 0.        , 0.        ,
        0.        , 0.        ]),
 3)

In [41]:
# 2020-08-30 Bokeh graph 3
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    # plot_height=300,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)

boka.circle(spec2.xs, spec2.y0s, color="red",size=2)
boka.multi_line(xBlPlot, yBlPlot, color="green")
boka.multi_line(xRgPlot, yRgPlot, color="orange")

# set up layout
p = gridplot([boka], ncols=1)
show(p)

In [42]:
# 2020-09-05 Bokeh graph 4
# blineMin - o melhor até agora
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
boka.circle(spec2.xs, spec2.y0s, color="red",size=2)
boka.line(spec2.xs, np.exp(blineMin), color="green")
# set up layout
p = gridplot([boka], ncols=1)
show(p)

In [43]:
# 2020-09-06 Bokeh graph 4b
# blineMin sobre lncounts com spline: *** directSpl ***- o melhorzíssimo até agora!!!!
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
boka.circle(spec2.xs, spec2.y0s, color="red",size=2)
boka.line(spec2.xs, directSpl, color="black")
# set up layout
p = gridplot([boka], ncols=1)
show(p)

In [44]:
# 2020-09-06 Bokeh graph 5
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    # plot_height=300,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
boka.circle(spec2.xs, spec2.y0s, color="red",size=2)
boka.line(spec2.xs, blineQuaPerc, color="orange")
p = gridplot([boka], ncols=1)
show(p)

In [45]:
# 2020-09-06 Bokeh graph 6
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    # plot_height=300,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
boka.circle(spec2.xs, spec2.y0s, color="red",size=2)
boka.line(spec2.xs, blineLinPerc, color="blue")
p = gridplot([boka], ncols=1)
show(p)

In [46]:
# 2020-09-06 Bokeh graph 7
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    # plot_height=300,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
boka.circle(spec2.xs, spec2.y0s, color="red",size=2)
boka.line(spec2.xs, blineConPerc, color="green")
p = gridplot([boka], ncols=1)
show(p)

In [47]:
PAREI AQUIIIIII 2020-09-05

SyntaxError: invalid syntax (<ipython-input-47-d8497885789c>, line 1)

In [48]:
nChPart

8192

In [49]:
is_bline = np.array(spec2.lnCounts)
for i,pk in enumerate(chanCwtpk):
    is_bline[pk-halfWins[i]:pk+halfWins[i]+1] = np.nan
mascara = ~(ma.masked_invalid(is_bline).mask)
xbline = spec2.xs[mascara]
ybline = spec2.lnCounts[mascara]

# turn UnivariateSpline(xbline, ybline, k=3, s=halfWin*10.0)


In [50]:
xs = spec2.xs[:nChPart]
ys = spec2.y0s[:nChPart]
lnNetCounts = np.zeros(nChPart)
uncys = np.sqrt(ys)
contin = np.zeros(nChPart)
yNets = np.zeros(nChPart)

In [51]:
splTck = splrep(xbline, ybline, k=1, s=smoo)
# tck é uma tupla com coeficientes, nós e grau
xnew = np.linspace(0, nChPart, num=20000)
ynewlog = np.exp(splev(xnew, splTck, der=0))

In [52]:
splTck = splrep(xbline, ybline, k=1, s=smoo)
# tck é uma tupla com coeficientes, nós e grau
xnew = np.linspace(0, nChPart, num=20000)
ynewlin = splev(xnew, splTck, der=0)

In [53]:
tck = splrep(xs, ys, s=smoo)
xnew = np.linspace(0, nChPart, num=1000)
ynew = splev(xnew, tck, der=0)

In [54]:
# 2020-08-30 Bokeh graph
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    plot_height=300,
    # plot_height=default_plot_height,
    tools=tools
)
boka.circle(indices_tupl[1], indices_tupl[0], size=3)

bokb = figure(
    plot_width =default_plot_width,
    plot_height=300,
    y_axis_type="log",
    tools=tools
)

bokb.x_range = boka.x_range
bokb.circle(spec2.xs, spec2.y0s, color="red",size=2)
bokb.line(spec2.xs, spec2.y0s, color="yellow")
bokb.circle(pksCh, spec2.y0s[pksCh], size=8, line_color='green', fill_color='white', fill_alpha=0)

bokb.circle(xbline, ybline, color="blue",size=2)
bokb.line(xbline, ybline, color="lightblue")

bokb.square(xnew, ynew, color="magenta",size=2)
bokb.line(xnew, ynew, color="orange")

# set up layout
p = gridplot([boka, bokb], ncols=1)
show(p)

In [55]:
spec2.bline_by_spline(pksCh)

In [56]:
smoo = nChPart - np.sqrt(2*nChPart)
smoo

8064.0

In [57]:
smoo=100

In [60]:
# 2020-08-30 Bokeh graph
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,wheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    plot_height=300,
    # plot_height=default_plot_height,
    tools=tools
)
boka.circle(indices_tupl[1], indices_tupl[0], size=3)

bokb = figure(
    plot_width =default_plot_width,
    plot_height=300,
    y_axis_type="log",
    tools=tools
)

bokb.x_range = boka.x_range
bokb.circle(spec2.xs, spec2.y0s, color="red",size=2)
bokb.line(spec2.xs, spec2.y0s, color="yellow")
bokb.circle(pksCh, spec2.y0s[pksCh], size=8, line_color='green', fill_color='white', fill_alpha=0)

bokb.circle(xnew, ynew, color="blue",size=2)
bokb.line(xnew, ynew, color="lightblue")

# set up layout
p = gridplot([boka, bokb], ncols=1)
show(p)

In [61]:
spec2.coeffs_2nd_grade_estim(haWin=haWin, hei=hei)

/home/marcelo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RankWarning: Polyfit may be poorly conditioned
  """Entry point for launching an IPython kernel.
/home/marcelo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RankWarning: Polyfit may be poorly conditioned
  """Entry point for launching an IPython kernel.


0

In [62]:
pks = spec2.ipks(widths_range=(5,15))
pks

array([ 112,  124,  237,  321,  379,  387,  392,  442,  470,  729,  781,
        828,  942, 1208, 1220, 1226, 1495, 1520, 1713, 1782, 1964, 1969,
       2034, 2039, 2052, 2073, 2119, 2161, 2297, 2303, 2343, 2379, 2433,
       2466, 2469, 2586, 2672, 2702, 2706, 2953, 3085, 3369, 3560, 3645,
       3682, 3760, 3890, 3979, 4025, 4082, 4247, 4249, 4357, 4614, 4730,
       4884, 4905, 4907, 5069, 5181, 5229, 5231, 5269, 5327, 5330, 5416,
       5418, 5461, 5674, 5851, 5987, 6117, 6271, 6489, 6560, 6610, 6620,
       6672, 6979, 7017, 7099, 7132, 7400, 7645, 7711, 7794, 7819, 8020,
       8045, 8076, 8101, 8180, 8187])

In [63]:
lnpks = spec2.lnIpks(widt=(5,20))
lnpks

array([ 112,  127,  237,  271,  321,  379,  386,  392,  426,  442,  464,
        470,  654,  727,  729,  781,  828,  942, 1060, 1208, 1217, 1226,
       1303, 1367, 1372, 1427, 1495, 1519, 1711, 1713, 1782, 1965, 1969,
       2034, 2044, 2052, 2073, 2119, 2159, 2303, 2343, 2379, 2433, 2466,
       2469, 2586, 2672, 2702, 2723, 2950, 2953, 3085, 3367, 3369, 3560,
       3645, 3682, 3758, 3760, 3890, 3976, 3979, 4025, 4082, 4183, 4244,
       4249, 4299, 4357, 4614, 4672, 4730, 4884, 4900, 4907, 5017, 5069,
       5228, 5269, 5327, 5332, 5415, 5418, 5462, 5674, 5742, 5851, 5987,
       5989, 6117, 6271, 6350, 6489, 6560, 6604, 6620, 6639, 6673, 6979,
       7017, 7100, 7132, 7400, 7645, 7711, 7794, 7803, 7819, 8020, 8045,
       8080, 8101, 8187])

In [64]:
lnpks = spec2.lnIpks(widt=(9,20), msnr=1.5)
lnpks

array([ 116,  127,  236,  321,  387,  464,  471,  729,  942, 1207, 1217,
       1226, 1368, 1372, 1495, 1713, 1782, 1965, 2033, 2044, 2076, 2159,
       2342, 2586, 2952, 3085, 3369, 3560, 3645, 3682, 3890, 3976, 3979,
       4082, 4244, 4249, 4357, 4614, 4730, 4900, 4907, 5069, 5228, 5328,
       5332, 5417, 5674, 5851, 5987, 6118, 6271, 6488, 6979, 7099, 7132,
       7400, 7645, 7793, 7803, 7821, 8020, 8074, 8173, 8183])

In [65]:
lnpks = spec2.lnIpks(widt=(7,8, 9,10,11,12), msnr=1.2)
lnpks

array([ 114,  236,  321,  388,  443,  471,  729,  781,  942, 1207, 1226,
       1368, 1495, 1713, 1782, 1965, 2033, 2052, 2075, 2301, 2342, 2432,
       2466, 2586, 2701, 2953, 3085, 3369, 3560, 3645, 3682, 3760, 3890,
       3979, 4025, 4082, 4249, 4357, 4614, 4730, 4883, 4907, 5069, 5228,
       5327, 5417, 5674, 5851, 5986, 6118, 6271, 6488, 6673, 6979, 7017,
       7100, 7132, 7400, 7645, 7794, 7820, 8020, 8045, 8074, 8185])

In [66]:
lnpks = spec2.lnIpks(widt=(5,6,7,8,9,10,11,12), msnr=1.2)
lnpks

array([ 112,  237,  321,  379,  388,  392,  426,  442,  470,  654,  729,
        781,  828,  942, 1208, 1226, 1367, 1495, 1519, 1713, 1782, 1965,
       1969, 2034, 2052, 2073, 2303, 2343, 2379, 2433, 2466, 2586, 2672,
       2702, 2953, 3085, 3369, 3560, 3645, 3682, 3760, 3890, 3979, 4025,
       4082, 4183, 4249, 4357, 4614, 4730, 4884, 4907, 5069, 5228, 5327,
       5418, 5674, 5851, 5987, 6117, 6271, 6489, 6560, 6607, 6673, 6979,
       7017, 7100, 7132, 7400, 7645, 7711, 7794, 7819, 8020, 8045, 8074,
       8101, 8187])

In [67]:
if fn2 == './I125_01.001.3.IEC':
    nChPart = 500
else:
    nChPart = spec2.lnCounts.size
nChPart

8192

In [68]:
smoo = nChPart - np.sqrt(2*nChPart)
# Aumentado o smooth:
smoo = 50*smoo
smoo

403200.0

In [69]:
ys[129:151]

array([923, 965, 907, 862, 776, 775, 787, 748, 822, 921, 922, 902, 862,
       782, 677, 733, 753, 762, 674, 700, 768, 745])

In [70]:
nContin = 4

In [71]:
xs = spec2.xs[:nChPart]
ys = spec2.y0s[:nChPart]
lnNetCounts = np.zeros(nChPart)
uncys = np.sqrt(ys)
contin = np.zeros(nChPart)
yNets = np.zeros(nChPart)

In [72]:
tck = splrep(xs, ys, s=smoo)
xnew = np.linspace(0, nChPart, num=2000)
ynew = splev(xnew, tck, der=0)

In [73]:
xnew

array([0.00000000e+00, 4.09804902e+00, 8.19609805e+00, ...,
       8.18380390e+03, 8.18790195e+03, 8.19200000e+03])

In [74]:
xnew.size

2000

In [75]:
ens = np.polyval(spec2.coeffs_ChEn, xs)
ensnew = np.polyval(spec2.coeffs_ChEn, xnew)
ensnew.size

2000

In [76]:
ynew

array([ 0.10274373,  0.02782861, -0.02129569, ..., 18.10512843,
       17.6097416 , 15.78302065])

In [77]:
x0 = splev([133], tck)[0]
x0

782.3174343416745

In [78]:
x1 = splev([147], tck)[0]
x1

679.0160012109075

In [79]:
pksLimits = np.array([[133,147],[151,168],[174,189]])

In [80]:
nPks = 3

In [81]:
for iPk in range(nPks):
    contin = stepBaseLine( pksLimits[iPk,0], pksLimits[iPk,1] )

TypeError: stepBaseLine() missing 3 required positional arguments: 'y0', 'y1', and 'ys'

In [ ]:
contin # 15, 18 e 16 canais incluindo os extremos

In [ ]:
yNets = ys - contin

In [ ]:
for i in range(nChPart):
    if yNets[i] < 2:
        lnNetCounts[i] = 0
    else:
        lnNetCounts[i] = np.log(yNets[i])

In [ ]:
coeffs_pks = np.zeros((nPks,3))
covars_pks = np.zeros((nPks,3,3))

In [ ]:
for iPk in range(nPks):
    xdata = xs[pksLimits[iPk,0]+3:pksLimits[iPk,1]-2]
    ydata = lnNetCounts[pksLimits[iPk,0]+3:pksLimits[iPk,1]-2]
    wYdata = uncys[pksLimits[iPk,0]+3:pksLimits[iPk,1]-2]
    if np.max(ydata) > 2.0:
        try:
            coeffs_pks[iPk], covars_pks[iPk] = np.polyfit(xdata, ydata, 2, w=wYdata, cov=True)
            # except LA.LinAlgError or ValueError:
        except:
            coeffs_pks[iPk] = np.zeros(3)
            covars_pks[iPk] = np.zeros((3,3))

In [ ]:
xdata

In [ ]:
ydata

In [ ]:
wYdata

In [ ]:
coeffs_pks

In [ ]:
covars_pks

In [ ]:
# 2020-08-15 AGORA interpretar os coefs. do ajuste gaussiano linearizado.

In [ ]:
# O vértice da parábola tem coord xV = -b/2a, y = c - b^2/4a.
# O fwhm será 2 delta x, onde delta x é x(H/2)-x(vertice)
# Usando a função quadrática inversa, será raiz(b^2) etc

# Os canais centroides e as alturas serão
chCtroids = np.zeros(nPks)
heights = np.zeros(nPks)
chFwhms = np.zeros(nPks)
for iPk in range(nPks):
    a = coeffs_pks[iPk,0]
    b = coeffs_pks[iPk,1]
    c = coeffs_pks[iPk,2]
    chCtroids[iPk] = -b / (2*a)
    heights[iPk] = np.exp( c - b**2 / (4*a))
    # Isso a seguir funciona mas é horrível...
    chFwhms[iPk] = np.sqrt( b**2 - 4*a*( c - np.log(heights[iPk]/2) ) ) / a
enCtroids = np.polyval(spec2.coeffs_ChEn, chCtroids)
print(chCtroids)
print(enCtroids)
print(heights)
print(chFwhms)

In [ ]:
# Perfeito!!!

In [ ]:
c

In [ ]:
# Gr'afico do ajuste:

In [ ]:
xnew_pks = np.zeros((nPks,130))
ynew_pks = np.zeros((nPks,130))
enNew_pks = np.zeros((nPks,130))
for iPk in range(nPks):
    xnew_pks[iPk] = np.linspace(pksLimits[iPk,0]+3, pksLimits[iPk,1]-3, num=130)
    ynew_pks[iPk] = np.exp(np.polyval(coeffs_pks[iPk], xnew_pks[iPk]))
    enNew_pks[iPk] = np.polyval(spec2.coeffs_ChEn, xnew_pks[iPk])

In [ ]:
xnew_pks[0]

In [ ]:
# PAREI AQUI a parte das paletas gr'aficas  2020-ago-12

In [ ]:
Paired6

In [ ]:
Pretos = ('#a6cee3', '#1f78b4', '#b2df8a', '#a6cee3', '#a6cee3', '#a6cee3')

In [ ]:
base, lower, upper = [], [], []

for i, x in enumerate(list(spec2.xs)):
    lower.append(spec2.y0s[i]-spec2.uncCounts[i])
    upper.append(spec2.y0s[i]+spec2.uncCounts[i])
    base.append(x)

source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

# Para nao cortar o numero da escala y:
yrange [1] = 1.5e6

bok1z = figure(
    plot_width = 900,
    # plot_width =default_plot_width,
    plot_height=default_plot_height,
    tools="pan,box_zoom,reset,save",
    y_axis_type="log",
    x_range=[22,39],
    y_range=yrange,
#    title=fn2
)
# bok1z.title.text = 'Click on legend entries to hide the corresponding lines'
bok1z.xaxis.axis_label = 'Energy (keV)'
bok1z.xaxis.axis_label_text_font_size = '18pt'
bok1z.xaxis.major_label_text_font_size = '16pt'

bok1z.yaxis.axis_label = 'Counts'
bok1z.yaxis.axis_label_text_font_size = '18pt'
bok1z.yaxis.major_label_text_font_size = '16pt'

base, lower, upper = [], [], []
for i, x in enumerate(list(ens)):
    lower.append(ys[i]-uncys[i])
    upper.append(ys[i]+uncys[i])
    base.append(x)
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

bok1z.circle(ens, ys, size=3, color='navy', alpha=1.0, legend_label='ys')
bok1z.line(ensnew, ynew, line_width=0.7, color='red', alpha=0.9, legend_label='gross')
bok1z.line(ens, contin, line_width=0.4, color='green', alpha=0.8, legend_label='contin')
bok1z.square(ens, yNets, size=7, line_color='navy', fill_color='orange', alpha=0.6, legend_label='net')
bok1z.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
) 

bok1z.legend.location = "top_right"
bok1z.legend.click_policy="hide"

bok1z.multi_line(list(enNew_pks), list(ynew_pks), line_width=0.3, color='blue', alpha=0.9, legend_label='pks')

mytext = Label(x=27.7, y=6.0e5, text='27 keV', text_font_size='18pt')
bok1z.add_layout(mytext)

mytext = Label(x=31.3, y=1.3e5, text='31 keV', text_font_size='18pt')
bok1z.add_layout(mytext)

mytext = Label(x=35.8, y=4.0e4, text='35 keV', text_font_size='18pt')
bok1z.add_layout(mytext)

show(bok1z)

In [ ]:
enNew_pks.shape

In [ ]:
# 2020-07-11 Bokeh graph w/ 2nd grade fit coefficients

uncys = np.sqrt(ys)

bokzz = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    tools="pan,box_zoom,reset,save",
    y_axis_type="log",
    x_range=[20,40],
    y_range=yrange,
)

bokzz.xaxis.axis_label = 'Energy (keV)'
bokzz.xaxis.axis_label_text_font_size = '16pt'
bokzz.xaxis.major_label_text_font_size = '14pt'

bokzz.yaxis.axis_label = 'Counts'
bokzz.yaxis.axis_label_text_font_size = '16pt'
bokzz.yaxis.major_label_text_font_size = '14pt'

base, lower, upper = [], [], []

for i, x in enumerate(list(ens)):
    lower.append(ys[i]-uncys[i])
    upper.append(ys[i]+uncys[i])
    base.append(x)

source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

bokzz.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)    
    
bokzz.circle(spec2.xs, spec2.y0s, color="red",size=4)
bokzz.line(spec2.xs, spec2.y0s)

show(bokzz)

In [ ]:
import pyplot as plt

In [ ]:
xdata = np.linspace(0, 4, 50)
y = func_example(xdata, 2.5, 1.3, 0.5)
np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')
popt, pcov = curve_fit(func_example, xdata, ydata)
popt

plt.plot(xdata, func_example(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
#popt, pcov = curve_fit(func_example, xdata, ydata, bounds=(0, [3., 1., 0.5]))
popt, pcov = curve_fit(func_example, xdata, ydata, bounds=([3., 1., 0.5], [3.0001, 1.0001, 0.5001]))
popt

plt.plot(xdata, func_example(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
xdata = np.linspace(0, 4, 50)
y = func_example(xdata, 2.5, 1.3, 0.5)
np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')
popt, pcov = curve_fit(func_example, xdata, ydata)
popt

plt.plot(xdata, func_example(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
popt, pcov = curve_fit(func_example, xdata, ydata, bounds=(0, [3., 1., 0.5]))
# popt, pcov = curve_fit(func_example, xdata, ydata, bounds=([3., 1., 0.5], [3.0001, 1.0001, 0.5001]))
popt

plt.plot(xdata, func_example(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
xdata = np.linspace(0, 4, 50)
y = func_example(xdata, 2.5, 1.3, 0.5)
np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')
popt, pcov = curve_fit(func_example, xdata, ydata)
popt

plt.plot(xdata, func_example(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
popt, pcov = curve_fit(func_example, xdata, ydata, bounds=(0, [3., 1., 0.5]))
# popt, pcov = curve_fit(func_example, xdata, ydata, bounds=([3., 1., 0.5], [3.0001, 1.0001, 0.5001]))
popt

plt.plot(xdata, func_example(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
fixed_b = 1.313

In [ ]:
xdata2 = np.concatenate((xdata, np.array([fixed_b])))

In [ ]:
# xdata = np.linspace(0, 4, 50)
# y = func_example(xdata, 2.5, 1.3, 0.5)
# np.random.seed(1729)
# y_noise = 0.2 * np.random.normal(size=xdata.size)
# ydata = y + y_noise

popt, pcov = curve_fit(func_example_fixed_b, xdata2, ydata)
tupla = (popt[0], popt[1], fixed_b) # QUELLE HORREUR

plt.plot(xdata, func_example_fixed_b(xdata2, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tupla)

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()
print(popt)